In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from evaluation.evaluation_harness import TrainEvaluator
from pprint import pprint

from dataset.data_loader import DataLoader, playtime_forever_users_games_edge_scoring_function, LinearNormalizer, GaussianNormalizer, PercentileNormalizer, never_remove_edge, remove_zero_playtime_edge
from models.random_model import RandomModel
from models.common_neighbors_model import CommonNeighbors
from models.popularity_model import GamePopularityModel
from models.ncf_model import NCFModel

In [2]:
data_loader = DataLoader(cache_local_dataset=True, users_games_edge_scoring_function = playtime_forever_users_games_edge_scoring_function, user_game_edge_embeddings = ['playtime_forever'], score_normalizers = [GaussianNormalizer(1.0, 1.0)])
data_loader.load_random_edge_train_test_split(train_percentage=0.8, test_percentage=0.2, seed=0)
data_loader.save_data_loader_parameters('test_evaluation_data_loader', overwrite=True)
evaluator = TrainEvaluator(data_loader, top_N_games_to_eval = 500, num_users_to_eval=10000, seed=0, debug=True)

In [4]:
random_model_initializer = lambda: RandomModel()
evaluator.reset(random_model_initializer, network_save_file = 'evaluation_test_random_model')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.plot_top_N_hit_percentage_percentiles(50)
evaluator.compute_top_N_hit_percentage(50)
evaluator.compute_top_N_recall(50)
evaluator.plot_top_N_recall_percentiles(50)
evaluator.save_metrics('test_random_model', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_test_random_model
Doen loading model. evaluation_test_random_model
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [03:02<00:00, 54.90it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
random
{'auc_roc': 0.43892371584260326,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 20.88192135711662,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.0003529251613792315,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.5003802848169288,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [3]:
popularity_model_initializer = lambda: GamePopularityModel()
evaluator.reset(popularity_model_initializer, network_save_file = 'evaluation_test_popularity_model')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.plot_top_N_hit_percentage_percentiles(50)
evaluator.compute_top_N_hit_percentage(50)
evaluator.compute_top_N_recall(50)
evaluator.plot_top_N_recall_percentiles(50)
evaluator.save_metrics('test_popularity_model', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_test_popularity_model
Doen loading model. evaluation_test_popularity_model
Getting predictions.


User Predictions:   0%|          | 0/10000 [00:00<?, ?it/s]

User Predictions: 100%|██████████| 10000/10000 [01:24<00:00, 117.69it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
game_popularity
{'auc_roc': 0.5515575916825022,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 18.282347335138635,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.03173301379601262,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.6601328958203395,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [5]:
common_neighbors_initializer = lambda: CommonNeighbors()
evaluator.reset(common_neighbors_initializer, network_save_file = 'evaluation_test_common_neighbors')
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.plot_top_N_hit_percentage_percentiles(50)
evaluator.compute_top_N_hit_percentage(50)
evaluator.compute_top_N_recall(50)
evaluator.plot_top_N_recall_percentiles(50)
evaluator.save_metrics('test_common_neighbors', overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Loading model: evaluation_test_common_neighbors
Doen loading model. evaluation_test_common_neighbors
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [11:49<00:00, 14.09it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
common_neighbors
{'auc_roc': 0.8613416577261556,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 17.23173562413738,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.029543197199454814,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.6511261844784793,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [5]:
save_file_name = 'evaluation_test_cf_low_weight_decay_increased_lr'
cf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='cf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(cf_initializer, network_save_file = save_file_name)
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.plot_top_N_hit_percentage_percentiles(50)
evaluator.compute_top_N_hit_percentage(50)
evaluator.compute_top_N_recall(50)
evaluator.plot_top_N_recall_percentiles(50)
evaluator.save_metrics(save_file_name, overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Known Game Embeddings:  []
Total Learnable Parameters: 5670100


Training: 100%|██████████| 40/40 [03:20<00:00,  5.02s/it]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [16:30<00:00, 10.10it/s] 


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_cf
{'auc_roc': 0.7326430988447226,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 13.750220204474886,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.0566,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.009512173397173574,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage': 0.02413,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall': 0.020276390819240136,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.5949171297573993,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [ ]:
save_file_name = 'evaluation_test_gcf_low_weight_decay_increased_lr'
gcf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [], seed=0, model_type='gcf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(gcf_initializer, network_save_file = )
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.plot_top_N_hit_percentage_percentiles(50)
evaluator.compute_top_N_hit_percentage(50)
evaluator.compute_top_N_recall(50)
evaluator.plot_top_N_recall_percentiles(50)
evaluator.save_metrics(save_file_name, overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Known Game Embeddings:  []
Total Learnable Parameters: 11340301


Training: 100%|██████████| 40/40 [10:18<00:00, 15.46s/it]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [10:22<00:00, 16.07it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_gcf
{'auc_roc': 0.7859937119787933,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 14.438437698831107,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.07046,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.011841479462276502,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage': 0.02995382027707834,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall': 0.02517028639036548,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.6121004029770417,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [ ]:
save_file_name = 'evaluation_test_mlp_low_weight_decay_increased_lr'
mlp_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 50, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [256, 256, 256], seed=0, model_type='mlp', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(mlp_initializer, network_save_file = save_file_name)
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.plot_top_N_hit_percentage_percentiles(50)
evaluator.compute_top_N_hit_percentage(50)
evaluator.compute_top_N_recall(50)
evaluator.plot_top_N_recall_percentiles(50)
evaluator.save_metrics(save_file_name, overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Known Game Embeddings:  []
Total Learnable Parameters: 11399081


Training: 100%|██████████| 40/40 [29:10<00:00, 43.76s/it]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [11:41<00:00, 14.25it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_mlp
{'auc_roc': 0.701227963598747,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 12.913123730707701,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.03959,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.006653479590001799,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage': 0.018936,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall': 0.01591188299018364,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.5801403293258488,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}



In [ ]:
save_file_name = 'evaluation_test_ncf_low_weight_decay_increased_lr'
ncf_initializer = lambda: NCFModel(num_epochs = 40, embedding_size = 25, batch_percent = 0.1, learning_rate = 1e-2, weight_decay=1e-6, mlp_hidden_layer_sizes = [256, 256, 256], seed=0, model_type='ncf', fine_tune_num_epochs=1, fine_tune_learning_rate=1e-1, fine_tune_weight_decay=1e-5, save_file_name=save_file_name, nn_save_name='best')
evaluator.reset(ncf_initializer, network_save_file = save_file_name)
evaluator.plot_top_N_recall_percentiles(10)
evaluator.plot_log_user_percentile_positional_error()
evaluator.compute_mean_positional_error()
evaluator.compute_top_N_recall(10)
evaluator.plot_roc_curve()
evaluator.compute_auc_roc()
evaluator.plot_user_rank_roc_curve()
evaluator.compute_user_rank_auc_roc()
evaluator.plot_top_N_hit_percentage_percentiles(10)
evaluator.compute_top_N_hit_percentage(10)
evaluator.plot_top_N_hit_percentage_percentiles(50)
evaluator.compute_top_N_hit_percentage(50)
evaluator.compute_top_N_recall(50)
evaluator.plot_top_N_recall_percentiles(50)
evaluator.save_metrics(save_file_name, overwrite=True)
print(evaluator.model.name())
pprint(evaluator.metrics)
print()

Preparing model.
Known Game Embeddings:  []
Total Learnable Parameters: 22739381


Training: 100%|██████████| 40/40 [1:24:47<00:00, 127.19s/it]


Done preparing model.
Getting predictions.


User Predictions: 100%|██████████| 10000/10000 [13:16<00:00, 12.55it/s]


Done getting predictions.
Appending dataframe information.
Ranking top N.
Done getting edge results.
neural_collborative_filtering_ncf
{'auc_roc': 0.692817107699023,
 'log_positional_error_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'mean_positional_error': 15.973808514978835,
 'roc_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_hit_percentage': 0.04527,
 'top_10_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_10_recall': 0.007608058121732291,
 'top_10_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_hit_percentage': 0.014183971632056735,
 'top_50_hit_percentage_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'top_50_recall': 0.011918786878578619,
 'top_50_recall_user_percentiles_figure': <Figure size 640x480 with 1 Axes>,
 'user_rank_auc_roc': 0.6395964476603843,
 'user_rank_roc_figure': <Figure size 640x480 with 1 Axes>}

